In [1]:
%reload_ext autoreload
%autoreload 2

from config import Config
from transcriptLoader import getCombinedTranscripts 

config = Config()
config.setFromEnv()

videoNames = [
    "New Quizzes Video",
    "Rearrange Playlist video",
    "IMSE 514 Presentation",
    "New Google Assignments in Canvas",
    "Piazza Introduction Workshop",
]

config.videoToUse = "New Google Assignments in Canvas"
transcriptToUse = getCombinedTranscripts(
    config.videoToUse,
    captionsFolder=config.captionsFolder,
)

In [3]:
from topicExtractor import retrieveTopics

topicsOverTime, topicModel = retrieveTopics(config.videoToUse, transcriptToUse, config=config)

Topic Model and Data loaded from saved files.
